# Step40, 브로드캐스트 함수 


## 40.1 broadcast_to 함수와 sum_to 함수(넘파이 버전)

넘파이의 np.broadcast_to(x,shape)을 살펴본다.  
ndarray 인스턴스인 x의 원소를 복제하여 shape 인수로 지정한 shape이 되도록 해준다.

In [7]:
import numpy as np 

x = np.array([1,2,3])
y = np.broadcast_to(x,(2,3))
print(y)
# (3,) -> (2,3)

[[1 2 3]
 [1 2 3]]


브로드캐스트('원소 복사')가 수행된 후의 역전파

'원소 복사'가 일어날 경우 역전파 때는 기울기의 '합'을 구한다.  
x는 \[1,2,3\] -> broadcaset_to -> y는 \[\[1,2,3\],\[1,2,3\]\]  
gx는 \[2,2,2\] <- sum_to <- gy는 \[\[1,1,1\],\[1,1,1\]\]

broadcaset_to 함수의 역전파는 입력 x의 형상과 같아지도록 기울기의 합을 구한다.  
sum_to는 x의 원소의 합을 구해 shape으로 만들어주는 함수이다.  
이런 함수로 순전파와 역전파의 관계를 만든다.

그러나 넘파이에는 이러한 함수가 없으므로 DeZero에서는 dezero/utils.py에 넘파이 버전 sum_to 함수를 준비해뒀다.

In [8]:
# utils.py
def sum_to(x, shape):
    ndim = len(shape)
    print(ndim)
    lead = x.ndim - ndim
    lead_axis = tuple(range(lead))
    print('lead:',lead)
    print('lead_axis',lead_axis)

    axis = tuple([i + lead for i, sx in enumerate(shape) if sx == 1])
    y = x.sum(lead_axis + axis, keepdims=True)
    if lead > 0:
        y = y.squeeze(lead_axis)
    return y

In [9]:
import numpy as np 
from dezero.utils import sum_to

x = np.array([[1,2,3],[4,5,6]])
y = sum_to(x,(1,3))
print(y)

y = sum_to(x,(2,1))
print(y)

# sum_to(x,shape) 함수는 shape 형상이 되도록 합을 계산한다. 

[[5 7 9]]
[[ 6]
 [15]]


## 40.2 broadcast_to 함수와 sum_to 함수(DeZero버전)

dezero/functions.py
~~~python 
class BroadcastTo(Function):
    def __init__(self,shape):
        self.shape = shape
    
    def forward(self, x):
        self.x_shape = x.shape 
        y = np.broadcast_to(x, self.shape)
        return y 
    
    def backward(self, gy):
        gx = sum_to(gy, self.x_shape)
        return gx 
    
def broadcast_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return BroadcastTo(shape)(x)
~~~

~~~python 
class SumTo(Function):
    def __init__(self, shape):
        self.shape = shape
    
    def forward(self, x):
        self.x_shape = x.shape
        y = utils.sum_to(x, self.shape)
        return y 
    
    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx 

def sum_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return SumTo(shape)(x)
~~~

DeZero의 broadcast_to 함수를 사용한다. 이처럼 broadcast_to 함수와 sum_to함수는 상호 의존적이다.

## 40.3 브로드캐스트 대응 

In [10]:
import numpy as np
from dezero.core import Variable
x0 = np.array([1,2,3])
x1 = np.array([10])
y = x0 + x1 
print(y)

[11 12 13]


In [11]:
x0 = Variable(np.array([1, 2, 3]))
x1 = Variable(np.array([10]))
y = x0 + x1 
print(y)

variable([11 12 13])


이와 같이 순전파는 ndarray 인스턴스를 사용해 구현했기 때문에 브로드캐스트가 일어난다.  
순전파에서 브로드캐스트가 일어났다면 그 역전파에서는 '브로드캐스트의 역전파'가 이루어져야 한다.  
하지만 현재 DeZero에서는 브로드캐스트의 역전파가 전혀 일어나지 않는다.

~~~python 
class Add(Function):
    def forward(self, x0, x1):
        self.x0_shape, self.x1_shape = x0.shape, x1.shape
        y = x0 + x1
        return y

    def backward(self, gy):
        gx0, gx1 = gy, gy
        if self.x0_shape != self.x1_shape:
            gx0 = dezero.functions.sum_to(gx0, self.x0_shape)
            gx1 = dezero.functions.sum_to(gx1, self.x1_shape)
        return gx0, gx1
~~~

순전파 때 브로드캐스트가 일어난다면, 입력되는 x0와 x1의 형상이 다를 것이다.  
이 점을 이용해 두 형상이 다를 때 브로드캐스트용 역전파를 계산하는 것이다.  
이를 위해 기울기 gx0는 x0의 형상이 되도록 합을 구하고, 마찬가지로 기울기 gx1은 x1의 형상이 되도록 합을 구한다.

core.py 에  
import dezero.functions 
추가



In [12]:
import numpy as np
from dezero import Variable

x0 = Variable(np.array([1, 2, 3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)


y.backward()
print(x1.grad)

variable([11 12 13])
variable([3])
